# flowspec与flow的区别
- SpiffWorkflow.specs.WorkflowSpec and SpiffWorkflow.specs.TaskSpec
    - workflowspec表示的整个所有可能的流程规范：它是通过链接所有taskspec到一棵树上来构成的
    - taskspec表示所有任务的规范
    - 他们都表示工作流定义
-  SpiffWorkflow.Workflow and SpiffWorkflow.Task.
    - workflow表示某一个具体的流程
    - task表示的是当前任务
    - 他们表示当前运行工作流状态的任务树建模。
    
# 工作流实现过程
## 定义工作流
- 使用spiffworkflow的各种taskspecs（比如Join，Split，Wait）以及由taskspecs派生出来的工作任务规范来定义工作流
- 规范的序列化和反序列化：
    - spiffworkflow有两种xml序列化器：
        - SpiffWorkflow.serializer.xml：标准化的工具，具有所有特性
        - SpiffWorkflow.serializer.prettyxml：对人类友好的工具，当手动编写xml时使用，不支持所有特性和序列化，可以用来反序列化
    - 定义示例：
            from SpiffWorkflow.specs import *
            from SpiffWorkflow import Workflow

            spec = WorkflowSpec()
            # (Add tasks to the spec here.)

            wf = Workflow(spec)
            wf.complete_task_from_id(...)
            
            加载xml文件来定义工作流
            from SpiffWorkflow.serializer.xml import XmlSerializer

            serializer = XmlSerializer()
            xml_file = 'my_workflow.xml'
            xml_data = open(xml_file).read()
            spec = serializer.deserialize_workflow_spec(xml_data, xml_file)
            
## 运行工作流
- 示例：
            from SpiffWorkflow import Workflow

            spec = ... # see above

            wf = Workflow(spec)
- Workflow对象表示正在运行的工作流的特定实例的状态。它通过保存一个由Task对象组成的树，包括派生树和数据。工作流的进度或状态中的所有更改总是反映在一个(或多个)Task对象中。每个Task都有一个状态，可以保存数据。
    - 为了是当前工作流可视化，可使用SpiffWorkflow.Workflow.dump()
    - 任务是由内部或外部条件的改变或者人为驱动，人为调用使用wf.complete_task_from_id(...)
    
## 认识任务状态
- MAYBE:表示任务在将来有可能会执行，但并不是必须
- LIKELY:和MAYBE类似，但是它有更高的可能性，而且它默认是任务下一个的选择
- FUTURE:表示处理器预测这条路径将被采用，而且这个任务必然在某一个点会被执行（除非这个任务取消，即CANCELED).如果一个任务是在等待前者的运行，那么它是FUTURE而不是WAITING状态
- WAITING:表示我正在处理属于我自己的任务，但是没有处理完。当任务处理完，我将是一个READY或者COMPLETED状态。它是一个可选状态
- READY:表示为满足该任务完成的前提条件已经具备
- COMPLETED:表四该任务已经完成
- CANECLED:任务取消

## taskspec data与task data
- taskspec data存储在TaskSpec对象中，当某一个任务改变了taskspec data中的数据，该更改将反映到使用TaskSpec对象的派生树中的所有其他实例。
- task data存储一个具体任务对象中，它会随着工作流的进行传递到它的子任务